In [6]:
import cv2
import numpy as np
import os

In [7]:
net = cv2.dnn.readNet("yolo_config/yolov3.weights", "yolo_config/yolov3.cfg")
print("YOLO model loaded successfully.")

# Get layer names
layer_names = net.getLayerNames()
print("Layer names:", layer_names)

classes = []
with open("yolo_config/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Get output layer names
output_layers_indices = net.getUnconnectedOutLayers()
output_layers = [layer_names[i - 1] for i in output_layers_indices]
print("Output layers:", output_layers)

YOLO model loaded successfully.
Layer names: ('conv_0', 'bn_0', 'leaky_1', 'conv_1', 'bn_1', 'leaky_2', 'conv_2', 'bn_2', 'leaky_3', 'conv_3', 'bn_3', 'leaky_4', 'shortcut_4', 'conv_5', 'bn_5', 'leaky_6', 'conv_6', 'bn_6', 'leaky_7', 'conv_7', 'bn_7', 'leaky_8', 'shortcut_8', 'conv_9', 'bn_9', 'leaky_10', 'conv_10', 'bn_10', 'leaky_11', 'shortcut_11', 'conv_12', 'bn_12', 'leaky_13', 'conv_13', 'bn_13', 'leaky_14', 'conv_14', 'bn_14', 'leaky_15', 'shortcut_15', 'conv_16', 'bn_16', 'leaky_17', 'conv_17', 'bn_17', 'leaky_18', 'shortcut_18', 'conv_19', 'bn_19', 'leaky_20', 'conv_20', 'bn_20', 'leaky_21', 'shortcut_21', 'conv_22', 'bn_22', 'leaky_23', 'conv_23', 'bn_23', 'leaky_24', 'shortcut_24', 'conv_25', 'bn_25', 'leaky_26', 'conv_26', 'bn_26', 'leaky_27', 'shortcut_27', 'conv_28', 'bn_28', 'leaky_29', 'conv_29', 'bn_29', 'leaky_30', 'shortcut_30', 'conv_31', 'bn_31', 'leaky_32', 'conv_32', 'bn_32', 'leaky_33', 'shortcut_33', 'conv_34', 'bn_34', 'leaky_35', 'conv_35', 'bn_35', 'leaky_36

In [8]:
model_name = "yolo3"
results = {}
for folders in os.listdir(f'data/frames'):
    output_directory = f'data/results/{model_name}/{folders}'
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    frames_info = {}
    folder_path = os.path.join('data/frames', folders)
    for j, file in enumerate(os.listdir(folder_path)):
        file_path = os.path.join(folder_path, file)
        frame = cv2.imread(file_path)
        height, width, channels = frame.shape

        # Detecting objects
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        # Initialize dictionary for this frame
        frame_info = {}

        # Showing informations on the screen
        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

                    # Add label and confidence to the frame info dictionary
                    label = str(classes[class_id])
                    frame_info[label] = confidence

        # Draw bounding boxes and labels on the frame
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        font = cv2.FONT_HERSHEY_PLAIN
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, label, (x, y + 30), font, 3, (0, 255, 0), 3)

        # Save the annotated frame
        annotated_frame_path = os.path.join(output_directory, f"result_{j}.jpg")
        cv2.imwrite(annotated_frame_path, frame)

        # Store frame info in the frames_info dictionary
        for k in frame_info.keys():
            if k in frames_info:
                frames_info[k] = max(frames_info[k], frame_info[k])
            else:
                frames_info[k] = [frame_info[k]]

    # Store frames_info in the results dictionary
    results[folders] = frames_info

# Save the results dictionary to a file
output_file = f"{model_name}/results.json"

print(results)

{'video_1': {'person': 0.99842525, 'tie': 0.98815, 'handbag': 0.8771827, 'sports ball': [0.5640002], 'car': [0.7938057], 'truck': 0.94684607}, 'video_2': {'person': 0.99818546, 'tie': 0.96933305, 'horse': 0.8338034, 'bus': 0.64566857, 'cup': [0.68669283], 'clock': 0.9106971, 'car': 0.9068541, 'bird': [0.548072], 'truck': 0.64088297, 'handbag': 0.7462338, 'aeroplane': [0.96137494]}, 'video_3': {'person': 0.99968964, 'tie': 0.9948254, 'horse': [0.72511107]}, 'video_4': {'bottle': 0.99435323, 'person': 0.99267805, 'cup': 0.83592176, 'diningtable': 0.7081988, 'suitcase': [0.73345166]}, 'video_5': {'book': [0.50674605], 'cup': 0.886211, 'toilet': [0.67074233], 'wine glass': 0.98148745, 'person': 0.91173065, 'boat': 0.9173018, 'surfboard': [0.71576685], 'bottle': 0.8308386}, 'video_6': {'person': 0.99602085, 'chair': [0.5383262], 'pottedplant': 0.8384062}, 'video_7': {'truck': [0.61620754], 'car': [0.6685091], 'person': 0.99582994, 'sofa': [0.73324287], 'teddy bear': [0.5316268]}, 'video_8':

In [9]:
for folder in results:
    print(folder)
    vr = results[folder]
    # sort the dictionary by value in descending order
    vr = dict(sorted(vr.items(), key=lambda item: item[1], reverse=True))
    for key in vr:
        print(key, vr[key])
        with open('data/results/' + model_name + '/' + folder + '/results.txt', 'a') as f:
            f.write(key + ': ' + str(vr[key]) + '\n')

video_1
person 0.99842525
tie 0.98815
truck 0.94684607
handbag 0.8771827
car [0.7938057]
sports ball [0.5640002]
video_2
person 0.99818546
tie 0.96933305
aeroplane [0.96137494]
clock 0.9106971
car 0.9068541
horse 0.8338034
handbag 0.7462338
cup [0.68669283]
bus 0.64566857
truck 0.64088297
bird [0.548072]
video_3
person 0.99968964
tie 0.9948254
horse [0.72511107]
video_4
bottle 0.99435323
person 0.99267805
cup 0.83592176
suitcase [0.73345166]
diningtable 0.7081988
video_5
wine glass 0.98148745
boat 0.9173018
person 0.91173065
cup 0.886211
bottle 0.8308386
surfboard [0.71576685]
toilet [0.67074233]
book [0.50674605]
video_6
person 0.99602085
pottedplant 0.8384062
chair [0.5383262]
video_7
person 0.99582994
sofa [0.73324287]
car [0.6685091]
truck [0.61620754]
teddy bear [0.5316268]
video_8
person 0.99773765
truck 0.9472315
tie 0.9205856
video_9
tie 0.99892133
person [0.99797225]
cup 0.97188395
truck 0.93341947
cat [0.77899945]
laptop [0.7685879]
bowl [0.7062782]
baseball bat [0.69448113]
